### **Week 15 복습과제**

https://github.com/CompVis/latent-diffusion/blob/main/ldm/modules/diffusionmodules/model.py

`ResnetBlock.forward`

In [ ]:
def forward(self, x, temb):
    # x: 현재 레이어의 입력 feature map (latent space에서의 representation)
    # temb: diffusion timestep t를 embedding한 벡터, 논문에서 말하는 "time conditioning"에 해당

    h = x

    # 첫 번째 normalization: latent feature를 안정화하여 diffusion step마다 분포 변화에 강건하게 만듦
    h = self.norm1(h)

    # SiLU / Swish 계열 nonlinearity - score-based diffusion model에서 표준적으로 사용
    h = nonlinearity(h)

    # 첫 번째 convolution: local spatial feature 추출 (UNet backbone의 기본 연산)
    h = self.conv1(h)

    if temb is not None:
        # timestep embedding을 projection하여 feature map에 더함
        h = h + self.temb_proj(nonlinearity(temb))[:, :, None, None]

    # 두 번째 normalization
    h = self.norm2(h)

    # nonlinearity
    h = nonlinearity(h)

    # dropout (diffusion 학습 시 과적합 방지)
    h = self.dropout(h)

    # 두 번째 convolution: residual block의 main transformation
    h = self.conv2(h)

    # shortcut (residual connection) 경로
    # 입력과 출력 channel 수가 다를 경우 dimension matching 필요
    if self.in_channels != self.out_channels:
        if self.use_conv_shortcut:
            # convolution shortcut
            x = self.conv_shortcut(x)
        else:
            # nin (1x1 convolution) shortcut
            x = self.nin_shortcut(x)

    # residual connection
    return x + h

`Model.forward`

In [ ]:
def forward(self, x, t=None, context=None):
    # x: 현재 diffusion step에서의 latent feature map
    # t: diffusion timestep (정수 index)
    # context: conditioning input (e.g., concat conditioning, spatial conditioning)

    # context가 주어지면 channel dimension으로 concat
    if context is not None:
        x = torch.cat((x, context), dim=1)

    # timestep conditioning 사용 여부
    if self.use_timestep:
        assert t is not None

        # diffusion timestep t를 sinusoidal embedding으로 변환
        temb = get_timestep_embedding(t, self.ch)

        # MLP를 통해 timestep embedding을 네트워크 내부 차원으로 projection
        temb = self.temb.dense[0](temb)
        temb = nonlinearity(temb)
        temb = self.temb.dense[1](temb)
    else:
        temb = None

    # 입력을 첫 convolution으로 latent feature로 변환
    # hs: skip connection을 저장하기 위한 리스트
    hs = [self.conv_in(x)]

    for i_level in range(self.num_resolutions):
        for i_block in range(self.num_res_blocks):
            # Residual block + timestep conditioning
            h = self.down[i_level].block[i_block](hs[-1], temb)

            # 해당 resolution에서 attention을 사용하는 경우
            if len(self.down[i_level].attn) > 0:
                h = self.down[i_level].attn[i_block](h)

            # skip connection으로 저장
            hs.append(h)

        # 마지막 resolution이 아니면 downsample 수행
        if i_level != self.num_resolutions - 1:
            hs.append(self.down[i_level].downsample(hs[-1]))

    # UNet의 가장 깊은 부분: 가장 global한 semantic representation
    h = hs[-1]

    # residual block + timestep conditioning
    h = self.mid.block_1(h, temb)

    # self-attention
    h = self.mid.attn_1(h)

    # 두 번째 residual block
    h = self.mid.block_2(h, temb)

    for i_level in reversed(range(self.num_resolutions)):
        for i_block in range(self.num_res_blocks + 1):
            # skip connection과 concat
            h = self.up[i_level].block[i_block](
                torch.cat([h, hs.pop()], dim=1), temb)

            # 해당 resolution에서 attention 사용
            if len(self.up[i_level].attn) > 0:
                h = self.up[i_level].attn[i_block](h)

        # 마지막 resolution이 아니면 upsample
        if i_level != 0:
            h = self.up[i_level].upsample(h)